**Задача 1:** Изучите как работает базовый reference counting

Создайте строку и исследуйте ее reference count на разных этапах. Верните кортеж: (count_after_creation, count_after_ref, count_after_del)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import sys
import gc
import weakref

In [2]:
def task1():
    # Ваш код здесь
    s = "Hello"
    count_after_creation = sys.getrefcount(s)

    ref = s
    count_after_ref = sys.getrefcount(s)

    del ref
    count_after_del = sys.getrefcount(s)

    return (count_after_creation, count_after_ref, count_after_del)

# Проверка
print("Задача 1:", task1())

Задача 1: (4, 5, 4)


**Задача 2:** Влияние функций на Reference Counting

Создайте список и передайте его в функцию.
Сравните reference count до и после вызова функции.
Верните кортеж: (count_before_call, count_during_call, count_after_call)

In [3]:
def task2():

    def process_list(lst):
        return sys.getrefcount(lst)

    # Ваш код здесь
    my_list = [1, 2, 3]
    count_before_call = sys.getrefcount(my_list) - 1

    count_during_call = process_list(my_list) - 1

    count_after_call = sys.getrefcount(my_list) - 1

    return (count_before_call, count_during_call, count_after_call)

# Проверка
print("Задача 2:", task2())

Задача 2: (1, 2, 1)


**Задача 3:** Циклические ссылки и Memory Leaks

Создайте два объекта с циклической ссылкой, затем разорвите ее.
Используйте gc для проверки количества собранных объектов.
Верните количество объектов собранных gc после разрыва ссылки.

In [6]:
def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    # Ваш код здесь
    node1 = Node("node1")
    node2 = Node("node2")
    node1.ref = node2
    node2.ref = node1

    node1.ref = None
    node2.ref = None

    collected_count = gc.collect()

    return collected_count

print("Задача 3:", task3())

Задача 3: 6


**Задача 4:** Сравнение Reference Count для разных типов данных

Сравните reference count для чисел, строк, списков и словарей.
Верните словарь с reference count для каждого типа после создания.

In [8]:
def task4():
    # Подсказка: можно посмотреть int, str, массив и другие
    # Ваш код здесь
    small_int_obj = 3
    big_int_obj = 2000
    float_obj = 3.14
    str_obj = "My_string"
    list_obj = [1, 2, 3]
    dict_obj = {'a': 1, 'b': 2}
    tuple_obj = (1, 2, 3)

    # Получаем reference count для каждого объекта
    ref_counts = {
        'cashed_int': sys.getrefcount(small_int_obj) - 1,
        'not_cashed_int' : sys.getrefcount(big_int_obj) - 1,
        'float': sys.getrefcount(float_obj) - 1,
        'str': sys.getrefcount(str_obj) - 1,
        'list': sys.getrefcount(list_obj) - 1,
        'dict': sys.getrefcount(dict_obj) - 1,
        'tuple': sys.getrefcount(tuple_obj) - 1
    }

    return ref_counts

print("Задача 4:", task4())

Задача 4: {'cashed_int': 4294967294, 'not_cashed_int': 3, 'float': 3, 'str': 2, 'list': 1, 'dict': 1, 'tuple': 2}


**Задача 5:** Weak References

Создайте два объекта со слабой ссылкой (weakref) между ними.
Убедитесь, что объекты могут быть удалены сборщиком мусора.
Верните True если weakref не увеличивает reference count.

In [9]:
def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    # Ваш код здесь
    data1 = Data(1)
    data2 = Data(2)

    weak_ref = weakref.ref(data2)
    data1.weak_ref = weak_ref

    ref_count_before = len(gc.get_referents(data2))

    del data2

    gc.collect()

    weak_ref_alive = weak_ref() is not None

    return not weak_ref_alive

print("Задача 5:", task5())

Задача 5: True


**Задача 6:** Мониторинг работы Garbage Collector

Зарегистрируйте callback функцию для отслеживания работы GC.
Верните список событий, которые отследил callback.

In [10]:
def task6():
    # Ваш код здесь
    events = []

    def gc_callback(phase, info):
        events.append({
            'phase': phase,
            'generation': info.get('generation'),
            'collections': info.get('collections'),
            'collected': info.get('collected'),
            'uncollectable': info.get('uncollectable')
        })

    gc.callbacks.append(gc_callback)
    gc.set_debug(gc.DEBUG_STATS)

    for i in range(1000):
        a = list(range(i))

    collected = gc.collect()
    gc.callbacks.remove(gc_callback)
    gc.set_debug(0)

    return events

print("Задача 6:", task6())

Задача 6: [{'phase': 'start', 'generation': 2, 'collections': None, 'collected': 0, 'uncollectable': 0}, {'phase': 'stop', 'generation': 2, 'collections': None, 'collected': 6, 'uncollectable': 0}]


gc: collecting generation 2...
gc: objects in each generation: 387 0 113628
gc: objects in permanent generation: 375
gc: done, 6 unreachable, 0 uncollectable, 0.0503s elapsed


**Задача 7:** Анализ поколений в GC

Создайте объекты и отследите их перемещение между поколениями GC.
Верните кортеж с количеством объектов в каждом поколении до и после создания объектов.

In [13]:
def task7():
    # Ваш код здесь
    initial_counts = gc.get_count()

    objects = []

    for i in range(1000):
        obj = list(range(i))
        objects.append(obj)

    counts_after_creation = gc.get_count()

    gc.collect(0)
    counts_after_gen0 = gc.get_count()

    gc.collect(1)
    counts_after_gen1 = gc.get_count()

    gc.collect(2)
    counts_after_full = gc.get_count()

    del objects
    gc.collect()

    return (initial_counts, counts_after_creation,
            counts_after_gen0, counts_after_gen1, counts_after_full)

print("Задача 7:", task7())

Задача 7: ((484, 0, 0), (67, 2, 0), (0, 3, 0), (0, 0, 1), (0, 0, 0))


**Задача 8:** Мониторинг порогов сборки мусора

Изучите как меняются счетчики поколений GC при создании объектов.
Верните словарь с состоянием счетчиков до и после создания объектов.

In [14]:
def task8():
    # Ваш код здесь, обратите внимание на return
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()

    objects = []

    for i in range(1500):
        obj = list(range(i))
        objects.append(obj)

    after_creation_count = gc.get_count()
    collected = gc.collect(0)
    after_collect_count = gc.get_count()

    del objects
    gc.collect()

    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }

print("Задача 8:", task8())

Задача 8: {'thresholds': (700, 10, 10), 'initial_count': (450, 0, 0), 'after_creation': (529, 2, 0), 'after_collect_gen0': (0, 3, 0), 'collected_objects': 0}


**Задача 9:** Быстрая проверка работы GC

Проверьте, собирает ли GC циклические ссылки.
Верните разницу в количестве объектов до и после сборки.

In [21]:
def task9():
    # Ваш код здесь
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    gc.disable()
    initial_objects = len(gc.get_objects())

    cycles = []
    for i in range(10):
        node1 = Node(f"node_{i}_a")
        node2 = Node(f"node_{i}_b")
        node1.ref = node2
        node2.ref = node1
        cycles.append((node1, node2))

    after_creation_objects = len(gc.get_objects())

    del cycles
    after_deletion_objects = len(gc.get_objects())
    collected = gc.collect()
    after_collection_objects = len(gc.get_objects())
    gc.enable()
    difference = after_deletion_objects - after_collection_objects

    return difference

print("Задача 9:", task9())

Задача 9: 64


In [ ]:
def task9_solution():
    # Создаем циклические ссылки быстро
    a, b, c = [], [], []
    a.append(b); b.append(c); c.append(a)  # Цикл a->b->c->a

    count_before = len(gc.get_objects())
    collected = gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after, collected

print("Задача 9:", task9_solution())

Задача 9: (10009, 10000)


**Задача 10:** Детектор "неубиваемых" объектов

Найдите объекты, которые переживают принудительную сборку мусора.
Выведите количество таких объектов.

In [15]:
def task10():
    # Ваш код здесь
    class Persistent:
        def __del__(self):
            pass  # Мешает нормальному удалению

    persistent_objects = []
    weak_refs = []

    # Создаем Persistent объекты (не удаляемые из-за __del__)
    for i in range(5):
        obj = Persistent()
        persistent_objects.append(obj)
        weak_refs.append(weakref.ref(obj))

    # Создаем циклические ссылки с Persistent
    p1 = Persistent()
    p2 = Persistent()
    p1.ref = p2
    p2.ref = p1
    weak_refs.append(weakref.ref(p1))
    weak_refs.append(weakref.ref(p2))

    del persistent_objects
    del p1, p2

    # Принудительная сборка мусора
    collected = gc.collect()

    # Проверяем, какие объекты выжили
    surviving_objects = 0
    for weak_ref in weak_refs:
        if weak_ref() is not None:
            surviving_objects += 1

    return surviving_objects

print("Задача 10:", task10())

Задача 10: 1


In [ ]:
# Небольшая подсказка для создания неудаляемых объектов
class Persistent:
        def __del__(self):
            pass  # Мешает нормальному удалению